<a href="https://colab.research.google.com/github/Narthana11/Assignment/blob/main/Lab4_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os; os.chdir('/content/drive/MyDrive/content')

In [ ]:
#Data Collection for The Assignment
import os
import csv

audio_dir = "audio"
csv_filename = "audio/dataset.csv"

with open(csv_filename, mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["filename", "label"])

    for label in os.listdir(audio_dir):
        label_dir = os.path.join(audio_dir, label)

        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                if filename.endswith(".wav"):
                    writer.writerow([filename, label])

print(f"CSV file '{csv_filename}' has been created successfully.")

CSV file 'audio/dataset.csv' has been created successfully.


In [ ]:
pip install torchmetrics

In [ ]:
!pip install sox

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import shutil

data_dir = "audio"
csv_file = "audio/dataset.csv"
output_dir = "trainingdata"

df = pd.read_csv(csv_file)

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

for dataset, dataset_df in zip(["train", "test"], [train_df, test_df]):
  for _, row in dataset_df.iterrows():
    filename = row['filename']
    label = row['label']

    source_path = os.path.join(data_dir, label, filename)

    destination_folder = os.path.join(output_dir, dataset, label)
    os.makedirs(destination_folder, exist_ok=True)
    destination_path = os.path.join(destination_folder, filename)
    shutil.copy(source_path, destination_path)

train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

In [ ]:
import torch
import torchaudio
from torch.utils.data import Dataset

class CustomSpeechDataset(Dataset):
    def __init__(self, csv_file, data_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        filename = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]

        audio_path = os.path.join(self.data_dir, label, filename)
        waveform, sample_rate = torchaudio.load(audio_path)

        if self.transform:
            waveform = self.transform(waveform)

        return waveform, sample_rate, label

train_dataset = CustomSpeechDataset(csv_file="split/train_labels.csv", data_dir="split/train/")
test_dataset = CustomSpeechDataset(csv_file="split/test_labels.csv", data_dir="split/test/")

print(f"Train Samples: {len(train_dataset)}, Test Samples: {len(test_dataset)}")


Train Samples: 192, Test Samples: 48


In [ ]:
import os
import torch
import torch.nn as nn  # Add this line
import torch.optim as optim
import torch.nn.functional as F






class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return x.squeeze(1)






In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, random_split
labels = set([d[2] for d in test_dataset])
label2num = {label:num for num, label in enumerate(labels)}


def collate_fn(batch):
  data = [b[0][0] for b in batch]
  data = pad_sequence(data, batch_first=True)
  labels = torch.LongTensor([label2num[b[2]] for b in batch])
  return data, labels


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


x, y = next(iter(train_loader))
x.shape, y





(torch.Size([32, 48000]),
 tensor([1, 3, 3, 0, 2, 0, 2, 1, 2, 3, 2, 1, 0, 2, 3, 2, 0, 1, 3, 0, 3, 1, 0, 1,
         2, 1, 0, 1, 0, 1, 0, 3]))

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [ ]:
from torchmetrics import Accuracy
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def train_one_epoch(model, train_loader, loss_fn, optimizer, epoch):
    model.train()
    loss_train = AverageMeter()

    acc_train = Accuracy(task="multiclass", num_classes=4).to(device)

    with tqdm(train_loader, unit="batch") as tepoch:
        for inputs, targets in tepoch:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            preds = outputs.argmax(dim=1)

            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()

            loss_train.update(loss.item(), inputs.size(0))
            acc_train.update(preds, targets)

            tepoch.set_postfix(loss=loss_train.avg, acc=acc_train.compute().item())

    return model, loss_train.avg, acc_train.compute().item()


In [ ]:
num_cls = 35

device = 'cuda' if torch.cuda.is_available() else 'cpu'
labels = set([d[2] for d in test_dataset])
label2num = {label:num for num, label in enumerate(labels)}
print(label2num)


{'unknown': 0, 'orange': 1, 'cherry': 2, 'apple': 3}


In [ ]:


def collate_fn(batch):
  data = [b[0][0] for b in batch]
  data = pad_sequence(data, batch_first=True)
  labels = torch.LongTensor([label2num[b[2]] for b in batch])
  return data, labels


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


x, y = next(iter(train_loader))
x.shape, y



(torch.Size([32, 48000]),
 tensor([0, 3, 3, 0, 1, 0, 1, 0, 3, 0, 0, 2, 2, 1, 2, 2, 1, 3, 3, 1, 0, 1, 3, 2,
         3, 0, 2, 2, 3, 2, 0, 1]))

In [ ]:
labels = set([d[2] for d in test_dataset])
label2num = {label:num for num, label in enumerate(labels)}
print(label2num)

{'unknown': 0, 'orange': 1, 'cherry': 2, 'apple': 3}


In [ ]:


def collate_fn(batch):
  data = [b[0][0] for b in batch]
  data = pad_sequence(data, batch_first=True)
  data = AF.resample(data, 16000, 8000).unsqueeze(1)
  labels = torch.LongTensor([label2num[b[2]] for b in batch])
  return data, labels


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


x, y = next(iter(train_loader))
x.shape, y



(torch.Size([32, 1, 24000]),
 tensor([3, 1, 2, 2, 0, 0, 1, 3, 1, 1, 0, 3, 1, 2, 1, 3, 0, 3, 3, 0, 3, 2, 2, 1,
         2, 0, 3, 0, 3, 2, 1, 1]))

In [ ]:
model = M5(n_input=1, n_output=num_cls).to(device)
loss_fn = nn.CrossEntropyLoss()

x_batch, y_batch = next(iter(train_loader))
outputs = model(x_batch.to(device))
loss = loss_fn(outputs, y_batch.to(device))
print(loss)

tensor(3.5067, grad_fn=<NllLossBackward0>)


In [ ]:
def validation(model, test_loader, loss_fn):
    model.eval()
    with torch.no_grad():
        loss_valid = AverageMeter()
        acc_test = Accuracy(task="multiclass", num_classes=4).to(device)

        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, targets)


            preds = outputs.argmax(dim=1)
            acc_test.update(preds, targets.int())

            loss_valid.update(loss.item())

    return loss_valid.avg, acc_test.compute().item()

In [ ]:


lr = 0.01
wd = 1e-3
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)


loss_train_hist = []
loss_test_hist = []

acc_train_hist = []
acc_test_hist = []

best_loss_test = torch.inf
epoch_counter = 0


num_epochs = 50

for epoch in range(num_epochs):
  model, loss_train, acc_train = train_one_epoch(model,
                                                 train_loader,
                                                 loss_fn,
                                                 optimizer,
                                                 epoch)
  loss_valid, acc_test = validation(model,
                                     test_loader,
                                     loss_fn)

  loss_train_hist.append(loss_train)
  loss_test_hist.append(loss_valid)

  acc_train_hist.append(acc_train)
  acc_test_hist.append(acc_test)

  if loss_valid < best_loss_valid:
    torch.save(model, f'model.pt')
    best_loss_valid = loss_valid
    print('Model Saved!')

  print(f'Valid: Loss = {loss_valid:.4}, Acc = {acc_test:.4}')
  print()

  epoch_counter += 1


100%|██████████| 6/6 [00:04<00:00,  1.30batch/s, acc=0.344, loss=2.54]


Model Saved!
Valid: Loss = 3.403, Acc = 0.25



100%|██████████| 6/6 [00:04<00:00,  1.21batch/s, acc=0.682, loss=1.81]


Model Saved!
Valid: Loss = 3.186, Acc = 0.25



100%|██████████| 6/6 [00:04<00:00,  1.49batch/s, acc=0.786, loss=1.36]


...

Valid: Loss = 0.4674, Acc = 0.8125



100%|██████████| 6/6 [00:04<00:00,  1.44batch/s, acc=1, loss=0.00537]


Valid: Loss = 0.4648, Acc = 0.8125



100%|██████████| 6/6 [00:04<00:00,  1.47batch/s, acc=1, loss=0.00528]


Valid: Loss = 0.4324, Acc = 0.7917

